1. 네이버 증권의 업종 분류를 가져와 적용시키는 곳

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime

In [2]:
# 오늘 날짜 가져오기
today = datetime.datetime.today()
today = today.strftime('%Y-%m-%d')
new_file_name = f'sector_info_df_{today}.csv'

In [6]:
sector_df = pd.read_csv(f'../../crawling/sector_info_df_2024-12-28.csv')


In [11]:
sector_df.head(1)

,업종코드,업종명,전일비,전체,상승,보합,하락,등락그래프
0,281,건강관리장비와용품,+1.17%,95,27,11,57,9%


In [7]:
def sector_detail_url(sector_name):
  '''
  업종명으로 해당업종의 상세목록을 볼수있는 url을 반환하는 함수
  :param sector_name: 업종명
  :return: 해당업종명에 해당하는 종목리스트를 볼수있는 주소
  '''
  sector_code = sector_df[sector_df['업종명'] == sector_name]['업종코드'].values[0]
  if sector_code.size == 0:
    print('존재하지 않는 업종입니다.')
    return None
  # print(type(sector_code))      #numpy.int64
  url = f'https://finance.naver.com/sise/sise_group_detail.naver?type=upjong&no={sector_code}'
  return url

In [8]:
sector_list = sector_df['업종명'].tolist()
sector_list

['건강관리장비와용품',
 '반도체와반도체장비',
 '종이와목재',
 '가정용품',
 '전기제품',
 '기타',
 '통신장비',
 '항공화물운송과물류',
 '양방향미디어와서비스',
 '게임엔터테인먼트',
 '레저용장비와제품',
 '포장재',
 '무선통신서비스',
 '전기장비',
 '전문소매',
 '전자장비와기기',
 '생물공학',
 '증권',
 '부동산',
 '다각화된통신서비스',
 '은행',
 '사무용전자제품',
 '제약',
 '판매업체',
 '손해보험',
 '기타금융',
 'IT서비스',
 '창업투자',
 '디스플레이장비및부품',
 '항공사',
 '섬유,의류,신발,호화품',
 '문구류',
 '화학',
 '컴퓨터와주변기기',
 '전자제품',
 '철강',
 '자동차',
 '도로와철도운송',
 '담배',
 '건설',
 '복합기업',
 '디스플레이패널',
 '식품',
 '석유와가스',
 '생명과학도구및서비스',
 '가정용기기와용품',
 '건강관리업체및서비스',
 '건축제품',
 '출판',
 '우주항공과국방',
 '조선',
 '기계',
 '핸드셋',
 '상업서비스와공급품',
 '자동차부품',
 '식품과기본식료품소매',
 '전기유틸리티',
 '소프트웨어',
 '화장품',
 '운송인프라',
 '가스유틸리티',
 '카드',
 '에너지장비및서비스',
 '호텔,레스토랑,레저',
 '해운사',
 '건축자재',
 '생명보험',
 '건강관리기술',
 '인터넷과카탈로그소매',
 '다각화된소비자서비스',
 '음료',
 '방송과엔터테인먼트',
 '무역회사와판매업체',
 '백화점과일반상점',
 '가구',
 '교육서비스',
 '광고',
 '비철금속',
 '복합유틸리티']

In [9]:
# 파일의 업종명을 돌면서, 해당 업종내 종목, 코스피/코스닥 유무 분류.
# 업종명에 접속->

sector_by_stock_list = []

for sector in sector_list :
   url = sector_detail_url(sector)
   res = requests.get(url)
   if res.status_code != 200:
     print('데이터를 가져오는데 실패했습니다.')
     exit()
   
   soup = BeautifulSoup(res.content, 'lxml')
   rows = soup.select('#contentarea > div:nth-child(5) > table > tbody > tr')
   
   for tr in rows :
     
    cols = tr.find_all("td")                        # BeautifulSoup 환경
    if len(cols) >= 9:
      stock_name = cols[0].text.strip()
      is_kosdaq = '*' in stock_name  # 별표가 있으면 True, 없으면 False
      stock_info = {
      '업종명': sector,  
      '종목명':stock_name.replace('*', '').strip(),  # * 기호 제거
      '코스닥 여부': is_kosdaq  # 코스닥 여부 추가
      }
      sector_by_stock_list.append(stock_info)

sector_by_stock_list_df = pd.DataFrame(sector_by_stock_list)
sector_by_stock_list_df.to_csv('sector_by_stock_list_df.csv', index=False, encoding='utf-8-sig')
display(sector_by_stock_list_df)

,업종명,종목명,코스닥 여부
0,건강관리장비와용품,나눔테크,False
1,건강관리장비와용품,타스컴,False
2,건강관리장비와용품,아이오바이오,False
3,건강관리장비와용품,넥스트바이오메디컬,True
4,건강관리장비와용품,진시스템,True
...,...,...,...
4210,비철금속,일진다이아,False
4211,비철금속,케이피에스,True
4212,비철금속,태경비케이,False
4213,비철금속,한컴위드,True


In [ ]:
# cols = tr.find_elements(By.TAG_NAME, "td")    # Selenium 환경